In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from tqdm import tqdm
from typing import List
import json
from concurrent.futures import ThreadPoolExecutor
from threading import Thread, local
from tqdm.contrib.concurrent import process_map
import csv

In [15]:
def to_csv(category: str) -> None:
    output_file = category + ".csv"
    if os.path.exists(output_file):
        return

    output: List[dict] = []
    for file in tqdm(os.listdir(category)):
        with open(os.path.join(category, file), "r") as reader:
            content = json.load(reader)
        for sample in content["content"]:
            output.append(
                {
                    "category": category,
                    "name": sample["name"],
                    "date": sample["date"],
                    "text": sample["text"],
                }
            )

    header = ["category", "name", "date", "text"]
    with open(output_file, "w") as file:
        writer = csv.DictWriter(file, fieldnames=header)
        writer.writeheader()
        for row in output:
            writer.writerow(row)

In [19]:
df = pd.read_csv("vetoes.csv")
df

TypeError: Cannot interpret '{'date', 'datetime64[h]'}' as a data type

In [14]:
for dir in tqdm(os.listdir(".")):
    if not os.path.isdir(dir):
        continue
    print(dir)
    to_csv(dir)

100%|██████████| 24/24 [07:04<00:00, 17.70s/it]


In [13]:
base_url = "https://www.presidency.ucsb.edu"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
}


def get_soup(url):
    page = requests.get(url, headers=headers)
    return BeautifulSoup(page.text, "html.parser")

In [14]:
MAX_WORKERS = 10

In [15]:
def get_soup(url):
    page = requests.get(url, headers=headers)
    return BeautifulSoup(page.text, "html.parser")

In [16]:
def from_last_page_to_all_url_pages(last_page_url: str) -> List[str]:
    first_page_url = "?".join(last_page_url.split("?")[:-1])
    nb_pages = int(last_page_url.split("=")[-1]) + 1
    all_urls: List[str] = [first_page_url]
    all_urls.extend(
        first_page_url + "?page=" + str(i)
        for i in tqdm(range(1, nb_pages), desc="Getting url_pages")
    )
    return all_urls

In [17]:
def _from_url_page_get_content_urls(url_page: str) -> List[str]:
    soup_page = get_soup(url_page)
    return [
        os.path.join(e["about"][1:])
        for e in soup_page.find_all(
            "div", "node node-documents node-teaser view-mode-teaser"
        )
    ]


def from_page_urls_to_content_urls(url_pages: List[str]) -> List[str]:
    content_urls = process_map(
        _from_url_page_get_content_urls,
        url_pages,
        max_workers=MAX_WORKERS,
        chunksize=MAX_WORKERS,
        desc="Getting content urls",
    )
    return [url for l in content_urls for url in l]

In [18]:
def _from_content_url_to_info_content(content_url: str) -> dict:
    new_url = os.path.join(base_url, content_url)
    sub_soup = get_soup(new_url)

    author = sub_soup.find("h3", class_="diet-title").text
    date = sub_soup.find("span", class_="date-display-single").text
    text = sub_soup.find("div", class_="field-docs-content").text
    return {"name": author, "date": date, "text": text}


def from_content_urls_to_infos_content(content_urls: List[str]) -> List[dict]:
    return process_map(
        _from_content_url_to_info_content,
        content_urls,
        max_workers=MAX_WORKERS,
        chunksize=MAX_WORKERS,
        desc="Getting info content",
    )

In [19]:
def save_infos_in_resources(infos: List[dict], category: str) -> None:
    dir_to_save = category
    os.makedirs(dir_to_save, exist_ok=True)
    for info in tqdm(infos, desc="Saving info"):
        title = info["name"] + "-" + info["date"] + ".json"
        content = {"category": category, "content": [info]}
        with open(os.path.join(dir_to_save, title), "w") as writer:
            json.dump(content, writer)

In [20]:
def _from_sub_section_to_last_page_url(url: str) -> str:
    print("Getting last page url")
    soup = get_soup(url)
    last = soup.find("li", class_="pager-last")
    return os.path.join(base_url, last.find("a")["href"][1:])


def from_sub_section_url_save_info(sub_section_url: str) -> None:
    category = sub_section_url.split("/")[-1]
    if category in os.listdir("."):
        print(f"Category '{category}' : Already in dataset")
        return

    print(f"Processing '{category}'")
    last_page_url = _from_sub_section_to_last_page_url(sub_section_url)
    url_pages = from_last_page_to_all_url_pages(last_page_url)
    content_urls = from_page_urls_to_content_urls(url_pages)
    infos = from_content_urls_to_infos_content(content_urls)
    save_infos_in_resources(infos, category)

In [21]:
def from_menu_to_sub_section_urls(url: str) -> List[str]:
    soup = get_soup(url)
    dropdown_menu = soup.find("li", class_="first expanded menu-mlid-10954 dropdown")
    return [
        os.path.join(base_url, child.find("a").attrs["href"][1:])
        for child in list(list(dropdown_menu.children)[1].children)
        if child != "\n"
    ]

In [22]:
url = "https://www.presidency.ucsb.edu/documents"
for sub_section_url in tqdm(from_menu_to_sub_section_urls(url), desc="Sub sections"):
    try:
        from_sub_section_url_save_info(sub_section_url)
    except KeyboardInterrupt as e:
        raise e
    except Exception as e:
        print(e.__traceback__)
        continue

Sub sections:   0%|          | 0/21 [00:00<?, ?it/s]

Category 'eulogies' : Already in dataset
Processing 'executive-orders'
Getting last page url


Getting content urls: 100%|██████████| 901/901 [01:45<00:00,  8.52it/s]
/tmp/ipykernel_26414/3393830896.py:12: TqdmWarning: Iterable length 9003 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  return process_map(
Sub sections:  10%|▉         | 2/21 [17:48<2:49:08, 534.13s/it]

Category 'fireside-chats' : Already in dataset
Processing 'interviews'
Getting last page url


Getting content urls: 100%|██████████| 103/103 [00:12<00:00,  8.52it/s]
/tmp/ipykernel_26414/3393830896.py:12: TqdmWarning: Iterable length 1028 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  return process_map(
Sub sections:  19%|█▉        | 4/21 [19:46<1:12:12, 254.84s/it]

...

Getting content urls: 100%|██████████| 1203/1203 [04:08<00:00,  4.85it/s]
/tmp/ipykernel_26414/3393830896.py:12: TqdmWarning: Iterable length 12029 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  return process_map(
Sub sections:  86%|████████▌ | 18/21 [3:05:37<26:09, 523.08s/it]

Processing 'vetoes'
Getting last page url


Getting content urls: 100%|██████████| 123/123 [00:19<00:00,  6.36it/s]
/tmp/ipykernel_26414/3393830896.py:12: TqdmWarning: Iterable length 1225 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  return process_map(
Sub sections:  90%|█████████ | 19/21 [3:07:58<14:16, 428.39s/it]

Category 'citations' : Already in dataset
Processing 'written-presidential-orders'
Getting last page url


Getting content urls: 100%|██████████| 2213/2213 [10:29<00:00,  3.51it/s]
/tmp/ipykernel_26414/3393830896.py:12: TqdmWarning: Iterable length 22124 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  return process_map(
Sub sections: 100%|██████████| 21/21 [3:56:02<00:00, 674.39s/it]
